In [ ]:
# Voila Web App

## A website built out of a Jupyter notebook using Voila

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
%matplotlib inline

In [1]:
import telegram
import pandas as pd
from selenium import webdriver
import numpy as np
import schedule
import datetime
import os
from time import sleep

In [2]:
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
from selenium.webdriver.chrome.options import Options

In [4]:
#options = Options()
#options.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
#driver = webdriver.Chrome(chrome_options = options, executable_path=r'/Users/sumitsingh/Desktop/chromedriver')
#driver.get('http://google.com/')
#print("Chrome Browser Invoked")
#driver.quit()

In [5]:
bot_token = '1982846194:AAFixZi2V9l3poBQQmSMm4FZ9y-mFTfryYA'
group_id = '@alert_nd_group'
bot = telegram.Bot(token=bot_token)

In [6]:
url = 'https://chartink.com/screener/sumit-test-1-inclusive'

In [7]:
def get_screened_stocks(url):

    options = Options()
    options.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
    driver = webdriver.Chrome(chrome_options = options, executable_path=r'/Users/sumitsingh/Desktop/chromedriver')    
    driver.get(url)
    table = pd.read_html(driver.find_element_by_xpath('//*[@id="DataTables_Table_0"]').get_attribute('outerHTML'))
    driver.close()
    return table[0]

In [8]:
kk = get_screened_stocks(url)

/var/folders/2g/qk_zllvd453cb3lmn3fl2tb00000gn/T/ipykernel_63760/3096846670.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = options, executable_path=r'/Users/sumitsingh/Desktop/chromedriver')


In [ ]:
kk.sort_values(by)

In [9]:
def job():
        screened_stocks = get_screened_stocks(url)
        screened_stocks.sort_values(by='Volume', ascending=0,inplace=True)
        screened_stocks_list = list(screened_stocks['Symbol'])
        bot.send_message(chat_id=group_id,text=f'HIKKAKE: {screened_stocks_list}')

In [ ]:
def job():
    
    if os.path.isfile(f'screened_stocks_{datetime.date.today()}.csv'):

        earlier_screened_stocks = pd.read_csv(f'screened_stocks_{datetime.date.today()}.csv')
        earlier_screened_stocks_list = list(earlier_screened_stocks['Symbol'])

        #again scan for screened stocks at sometime if the file .csv of todays date already exists
        screened_stocks = get_screened_stocks(url)
        screened_stocks.sort_values(by='Volume', ascending=0,inplace=True)
        screened_stocks_list = list(screened_stocks['Symbol'])

        new_screened_stocks_list = []
        for stock in screened_stocks_list:
            if stock not in earlier_screened_stocks_list:
                new_screened_stocks_list.append(stock)

        for stock in new_screened_stocks_list:
                bot.send_message(chat_id=group_id,text=f'HIKKAKE: {stock}')
                
        screened_stocks = pd.concat([earlier_screened_stocks,screened_stocks],axis=0)
        screened_stocks = screened_stocks.drop_duplicates(subset=['Symbol'])
        screened_stocks.to_csv(f'screened_stocks_{datetime.date.today()}.csv',index=False)

    else:
        screened_stocks = get_screened_stocks(url)
        screened_stocks.sort_values(by='Volume', ascending=0,inplace=True)
        screened_stocks_list = list(screened_stocks['Symbol'])
        screened_stocks.to_csv(f'screened_stocks_{datetime.date.today()}.csv',index=False)

        for stock in screened_stocks_list:
                bot.send_message(chat_id=group_id,text=f'HIKKAKE: {stock}')

In [14]:
while True:
    
    if (datetime.date.today().weekday() != 6) & (datetime.date.today().weekday() != 5):

        hours = [9,10,11,12,13,14,15,23]
        minutes = [0,15,30,45,4]

        for hour in hours:
            for minute in minutes:
                schedule_time = datetime.time(hour,minute)
                if (schedule_time >= datetime.time(9,15)) & (schedule_time <= datetime.time(15,40)):
                    schedule.every().day.at(str(schedule_time)).do(job)

        while True:
            schedule.run_pending()
            sleep(14*60)

    else:

        now_datetime = datetime.datetime.now()
        next_datetime = datetime.datetime.combine(datetime.date(now_datetime.year,now_datetime.month,now_datetime.day + 1),datetime.time(9,15))
        sleep((next_datetime - now_datetime).seconds)

KeyboardInterrupt: 

In [12]:
sleep?